In [17]:
import pickle
import IPython.display as ipd
from sklearn.decomposition import PCA
import librosa
import numpy as np
import glob
import os

In [6]:

ipd.Audio('devel_0001.wav')

In [7]:

data, sampling_rate = librosa.load('devel_0001.wav')

In [4]:

mfccs = np.mean(librosa.feature.mfcc(y=data, sr=sampling_rate, n_mfcc=40).T,axis=0) 

In [8]:
labels = {}
mapping ={"angry":0,"neutral":1,"happy":2,"sad":3}
with open("ComParE2018_AtypicalAffect.txt") as f1:
    for line in f1:
        key,value = line.split()
        if value in mapping:
            labels[key] = mapping[value]
        

In [9]:
len(labels.keys())

7341

In [11]:
def get_feature(filename,feature):
    data, sampling_rate = librosa.load(filename)
    out_feature = np.mean(feature(y=data).T,axis=0)
    return out_feature

In [12]:
features_list =[librosa.feature.chroma_cqt,librosa.feature.chroma_cens,librosa.feature.melspectrogram,librosa.feature.chroma_stft,\
               librosa.feature.rms,librosa.feature.mfcc,librosa.feature.rmse, librosa.feature.spectral_centroid, \
               librosa.feature.spectral_bandwidth,librosa.feature.spectral_contrast,librosa.feature.spectral_flatness,\
               librosa.feature.spectral_rolloff, librosa.feature.poly_features,librosa.feature.tonnetz,\
               librosa.feature.zero_crossing_rate,librosa.feature.tempogram]

In [13]:
def get_data(feature):
    train_data = []
    train_labels = []
    test_data = []
    test_labels =[]
    valid_data = []
    valid_labels =[]
    cwd = os.getcwd()
    path = cwd + "/wav-20190419T185205Z-001/wav/"
    for filename in glob.glob(os.path.join(path,'*.wav')):
        temp_key = filename.rsplit('/',1)[1]
        if(temp_key.startswith("train")):
            train_data.append(get_feature(filename,feature))
            train_labels.append(labels[temp_key])
        elif (temp_key.startswith("test")):
            test_data.append(get_feature(filename,feature))
            test_labels.append(labels[temp_key])
        else:
            valid_data.append(get_feature(filename,feature))
            valid_labels.append(labels[temp_key])
    return train_data,train_labels,test_data,test_labels,valid_data,valid_labels

In [14]:
def get_stats(feature_list):
    dict1 ={}
    for feature in feature_list:
        train_data,train_labels,test_data,test_labels,valid_data,valid_labels = get_data(feature)
        acc = get_accuracy(train_data,train_labels,test_data,test_labels,valid_data,valid_labels)
        dict1[feature] = acc
        print(feature," ",acc)
    return dict1

In [15]:
from sklearn.svm import SVC
def get_accuracy(train_data,train_labels,test_data,test_labels,valid_data,valid_labels):
    clf = SVC()
    clf.fit(train_data,train_labels)
    return clf.score(test_data,test_labels)

In [18]:
def get_data_dict(features_list):
    data = {}
    cwd = os.getcwd()
    path = cwd + "/wav-20190419T185205Z-001/wav/"
    for filename in glob.glob(os.path.join(path,'*.wav')):
        key = filename.rsplit('/',1)[1]
        if key not in data:
            data[key] = []
            for feature in features_list:
                temp_list =  get_feature(filename,feature)
                data[key].extend(temp_list)
                
    return data    

In [19]:
def get_acc_complete_features(feature_list):
    train_data =   []
    train_labels = []
    test_data  =   []
    test_labels =  []
    valid_data  =  []
    valid_labels = []
    data_dict = get_data_dict(feature_list)
    with open('data.pickle', 'wb') as handle:
        pickle.dump(data_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
    for key in data_dict.keys():
        if key.startswith("train"):
            train_data.append(data_dict[key])
            train_labels.append(labels[key])
        if key.startswith("test"):
            test_data.append(data_dict[key])
            test_labels.append(labels[key])
        if key.startswith("devel"):
            valid_data.append(data_dict[key])
            valid_labels.append(labels[key])
    return train_data,train_labels,test_data,test_labels,valid_data,valid_labels

In [20]:
train_data,train_labels,test_data,test_labels,valid_data,valid_labels = get_acc_complete_features(features_list)
acc = get_accuracy(train_data,train_labels,test_data,test_labels,valid_data,valid_labels)
print(acc)

/home/saieswar/anaconda2/envs/cs633/lib/python3.6/site-packages/scipy/fftpack/basic.py:160: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  z[index] = x
/home/saieswar/anaconda2/envs/cs633/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.6916916916916916


In [24]:
valid_data =   []
valid_labels = []
train_data  =  []
train_labels = []
test_data =    []
test_labels =  [] 
import librosa
import glob
import numpy as np
import os
count =0
cwd = os.getcwd()
path = cwd + "/wav-20190419T185205Z-001/wav/"
for filename in glob.glob(os.path.join(path, 'devel*.wav')):
    data, sampling_rate = librosa.load(filename)
    mfccs = np.mean(librosa.feature.mfcc(y=data, sr=sampling_rate, n_mfcc=10).T,axis=0)
    temp_key = filename.rsplit('/',1)[1]
    print(temp_key," ",count)
    count +=1
    temp_label = labels[temp_key]
    valid_data.append(mfccs)
    valid_labels.append(temp_label)

for filename in glob.glob(os.path.join(path, 'train*.wav')):
    data, sampling_rate = librosa.load(filename)
    mfccs = np.mean(librosa.feature.mfcc(y=data, sr=sampling_rate, n_mfcc=10).T,axis=0)
    temp_key = filename.rsplit('/',1)[1]
    print(temp_key," ",count)
    count +=1
    print(temp_key)
    temp_label = labels[temp_key]
    train_data.append(mfccs)
    train_labels.append(temp_label)


for filename in glob.glob(os.path.join(path, 'test*.wav')):
    data, sampling_rate = librosa.load(filename)
    mfccs = np.mean(librosa.feature.mfcc(y=data, sr=sampling_rate, n_mfcc=10).T,axis=0)
    temp_key = filename.rsplit('/',1)[1]
    print(temp_key," ",count)
    count +=1
    print(temp_key)
    temp_label = labels[temp_key]
    test_data.append(mfccs)
    test_labels.append(temp_label)

devel_2395.wav   0
devel_1377.wav   1
devel_2500.wav   2
devel_1926.wav   3
devel_1622.wav   4
devel_2096.wav   5
devel_1724.wav   6
devel_1447.wav   7
devel_2589.wav   8
devel_2600.wav   9
devel_1766.wav   10
devel_0459.wav   11
devel_0765.wav   12
devel_0835.wav   13
devel_1568.wav   14
devel_2411.wav   15
devel_2751.wav   16
devel_1169.wav   17
devel_1505.wav   18
devel_2938.wav   19
devel_1243.wav   20
devel_2427.wav   21
devel_0010.wav   22
devel_0945.wav   23
devel_1781.wav   24
devel_1006.wav   25
devel_1918.wav   26
devel_2694.wav   27
devel_2220.wav   28
devel_0397.wav   29
devel_0270.wav   30
devel_1772.wav   31
devel_0650.wav   32
devel_0828.wav   33
devel_2016.wav   34
devel_0806.wav   35
devel_1411.wav   36
devel_0857.wav   37
devel_2468.wav   38
devel_0393.wav   39
devel_2032.wav   40
devel_1981.wav   41
devel_1209.wav   42
devel_1186.wav   43
devel_0316.wav   44
devel_1218.wav   45
devel_0325.wav   46
devel_1656.wav   47
devel_2246.wav   48
devel_2664.wav   49
devel_1902

devel_2027.wav   398
devel_2128.wav   399
devel_0391.wav   400
devel_1626.wav   401
devel_2658.wav   402
devel_2817.wav   403
devel_0973.wav   404
devel_2394.wav   405
devel_1996.wav   406
devel_1726.wav   407
devel_0263.wav   408
devel_0881.wav   409
devel_1246.wav   410
devel_1844.wav   411
devel_2180.wav   412
devel_0150.wav   413
devel_1429.wav   414
devel_1698.wav   415
devel_1283.wav   416
devel_2581.wav   417
devel_1286.wav   418
devel_2261.wav   419
devel_2966.wav   420
devel_0120.wav   421
devel_0180.wav   422
devel_2435.wav   423
devel_0599.wav   424
devel_2031.wav   425
devel_2984.wav   426
devel_0258.wav   427
devel_2013.wav   428
devel_2605.wav   429
devel_0012.wav   430
devel_0972.wav   431
devel_2584.wav   432
devel_1100.wav   433
devel_2476.wav   434
devel_1661.wav   435
devel_0328.wav   436
devel_2877.wav   437
devel_2111.wav   438
devel_0484.wav   439
devel_0960.wav   440
devel_0239.wav   441
devel_2723.wav   442
devel_2185.wav   443
devel_0342.wav   444
devel_0862.wa

devel_1227.wav   789
devel_2017.wav   790
devel_0673.wav   791
devel_1526.wav   792
devel_2728.wav   793
devel_1686.wav   794
devel_1583.wav   795
devel_0388.wav   796
devel_1128.wav   797
devel_0909.wav   798
devel_0371.wav   799
devel_0778.wav   800
devel_2482.wav   801
devel_0823.wav   802
devel_0919.wav   803
devel_1455.wav   804
devel_0472.wav   805
devel_0576.wav   806
devel_1493.wav   807
devel_1117.wav   808
devel_2946.wav   809
devel_2277.wav   810
devel_1903.wav   811
devel_1961.wav   812
devel_1244.wav   813
devel_1807.wav   814
devel_2315.wav   815
devel_1123.wav   816
devel_0142.wav   817
devel_2983.wav   818
devel_2260.wav   819
devel_1033.wav   820
devel_2347.wav   821
devel_0888.wav   822
devel_0085.wav   823
devel_2540.wav   824
devel_1307.wav   825
devel_0518.wav   826
devel_0523.wav   827
devel_1580.wav   828
devel_1336.wav   829
devel_2509.wav   830
devel_0501.wav   831
devel_1971.wav   832
devel_1594.wav   833
devel_0158.wav   834
devel_1428.wav   835
devel_1437.wa

devel_0352.wav   1172
devel_1706.wav   1173
devel_2539.wav   1174
devel_0602.wav   1175
devel_0108.wav   1176
devel_1045.wav   1177
devel_0290.wav   1178
devel_0494.wav   1179
devel_2571.wav   1180
devel_0799.wav   1181
devel_0031.wav   1182
devel_1574.wav   1183
devel_2813.wav   1184
devel_2812.wav   1185
devel_0918.wav   1186
devel_2599.wav   1187
devel_2631.wav   1188
devel_1832.wav   1189
devel_2744.wav   1190
devel_1535.wav   1191
devel_0475.wav   1192
devel_0734.wav   1193
devel_1791.wav   1194
devel_1523.wav   1195
devel_0816.wav   1196
devel_0453.wav   1197
devel_1050.wav   1198
devel_1673.wav   1199
devel_2007.wav   1200
devel_0729.wav   1201
devel_2155.wav   1202
devel_0667.wav   1203
devel_1438.wav   1204
devel_1645.wav   1205
devel_0347.wav   1206
devel_0412.wav   1207
devel_2890.wav   1208
devel_1649.wav   1209
devel_2167.wav   1210
devel_0219.wav   1211
devel_2900.wav   1212
devel_0244.wav   1213
devel_0121.wav   1214
devel_1342.wav   1215
devel_1871.wav   1216
devel_0418

devel_2580.wav   1546
devel_2675.wav   1547
devel_2833.wav   1548
devel_2076.wav   1549
devel_0512.wav   1550
devel_2747.wav   1551
devel_0127.wav   1552
devel_2716.wav   1553
devel_2491.wav   1554
devel_1434.wav   1555
devel_1260.wav   1556
devel_0846.wav   1557
devel_2666.wav   1558
devel_1504.wav   1559
devel_0103.wav   1560
devel_0370.wav   1561
devel_1489.wav   1562
devel_1173.wav   1563
devel_0910.wav   1564
devel_2229.wav   1565
devel_1303.wav   1566
devel_1360.wav   1567
devel_2236.wav   1568
devel_0874.wav   1569
devel_2061.wav   1570
devel_0481.wav   1571
devel_1723.wav   1572
devel_1537.wav   1573
devel_2657.wav   1574
devel_1578.wav   1575
devel_1104.wav   1576
devel_0766.wav   1577
devel_1016.wav   1578
devel_2980.wav   1579
devel_2507.wav   1580
devel_1355.wav   1581
devel_1311.wav   1582
devel_1694.wav   1583
devel_0125.wav   1584
devel_0234.wav   1585
devel_1506.wav   1586
devel_0502.wav   1587
devel_0745.wav   1588
devel_2820.wav   1589
devel_2142.wav   1590
devel_2801

...

test_0997.wav   6364
test_0997.wav
test_0237.wav   6365
test_0237.wav
test_0033.wav   6366
test_0033.wav
test_0861.wav   6367
test_0861.wav
test_0725.wav   6368
test_0725.wav
test_0869.wav   6369
test_0869.wav
test_0713.wav   6370
test_0713.wav
test_0201.wav   6371
test_0201.wav
test_0888.wav   6372
test_0888.wav
test_0495.wav   6373
test_0495.wav
test_0006.wav   6374
test_0006.wav
test_0191.wav   6375
test_0191.wav
test_0361.wav   6376
test_0361.wav
test_0714.wav   6377
test_0714.wav
test_0650.wav   6378
test_0650.wav
test_0030.wav   6379
test_0030.wav
test_0612.wav   6380
test_0612.wav
test_0348.wav   6381
test_0348.wav
test_0080.wav   6382
test_0080.wav
test_0263.wav   6383
test_0263.wav
test_0088.wav   6384
test_0088.wav
test_0149.wav   6385
test_0149.wav
test_0177.wav   6386
test_0177.wav
test_0516.wav   6387
test_0516.wav
test_0999.wav   6388
test_0999.wav
test_0726.wav   6389
test_0726.wav
test_0067.wav   6390
test_0067.wav
test_0877.wav   6391
test_0877.wav
test_0885.wav   6392

test_0101.wav   6600
test_0101.wav
test_0744.wav   6601
test_0744.wav
test_0304.wav   6602
test_0304.wav
test_0746.wav   6603
test_0746.wav
test_0605.wav   6604
test_0605.wav
test_0638.wav   6605
test_0638.wav
test_0204.wav   6606
test_0204.wav
test_0823.wav   6607
test_0823.wav
test_0593.wav   6608
test_0593.wav
test_0973.wav   6609
test_0973.wav
test_0912.wav   6610
test_0912.wav
test_0528.wav   6611
test_0528.wav
test_0622.wav   6612
test_0622.wav
test_0383.wav   6613
test_0383.wav
test_0032.wav   6614
test_0032.wav
test_0960.wav   6615
test_0960.wav
test_0923.wav   6616
test_0923.wav
test_0419.wav   6617
test_0419.wav
test_0111.wav   6618
test_0111.wav
test_0717.wav   6619
test_0717.wav
test_0936.wav   6620
test_0936.wav
test_0940.wav   6621
test_0940.wav
test_0838.wav   6622
test_0838.wav
test_0561.wav   6623
test_0561.wav
test_0238.wav   6624
test_0238.wav
test_0964.wav   6625
test_0964.wav
test_0332.wav   6626
test_0332.wav
test_0397.wav   6627
test_0397.wav
test_0993.wav   6628

test_0484.wav   6837
test_0484.wav
test_0673.wav   6838
test_0673.wav
test_0710.wav   6839
test_0710.wav
test_0913.wav   6840
test_0913.wav
test_0169.wav   6841
test_0169.wav
test_0916.wav   6842
test_0916.wav
test_0365.wav   6843
test_0365.wav
test_0274.wav   6844
test_0274.wav
test_0815.wav   6845
test_0815.wav
test_0422.wav   6846
test_0422.wav
test_0235.wav   6847
test_0235.wav
test_0278.wav   6848
test_0278.wav
test_0002.wav   6849
test_0002.wav
test_0557.wav   6850
test_0557.wav
test_0738.wav   6851
test_0738.wav
test_0062.wav   6852
test_0062.wav
test_0015.wav   6853
test_0015.wav
test_0221.wav   6854
test_0221.wav
test_0014.wav   6855
test_0014.wav
test_0760.wav   6856
test_0760.wav
test_0524.wav   6857
test_0524.wav
test_0579.wav   6858
test_0579.wav
test_0938.wav   6859
test_0938.wav
test_0466.wav   6860
test_0466.wav
test_0193.wav   6861
test_0193.wav
test_0552.wav   6862
test_0552.wav
test_0664.wav   6863
test_0664.wav
test_0770.wav   6864
test_0770.wav
test_0549.wav   6865

test_0687.wav   7073
test_0687.wav
test_0900.wav   7074
test_0900.wav
test_0199.wav   7075
test_0199.wav
test_0359.wav   7076
test_0359.wav
test_0384.wav   7077
test_0384.wav
test_0920.wav   7078
test_0920.wav
test_0460.wav   7079
test_0460.wav
test_0171.wav   7080
test_0171.wav
test_0615.wav   7081
test_0615.wav
test_0155.wav   7082
test_0155.wav
test_0843.wav   7083
test_0843.wav
test_0604.wav   7084
test_0604.wav
test_0048.wav   7085
test_0048.wav
test_0686.wav   7086
test_0686.wav
test_0978.wav   7087
test_0978.wav
test_0755.wav   7088
test_0755.wav
test_0190.wav   7089
test_0190.wav
test_0077.wav   7090
test_0077.wav
test_0660.wav   7091
test_0660.wav
test_0513.wav   7092
test_0513.wav
test_0163.wav   7093
test_0163.wav
test_0932.wav   7094
test_0932.wav
test_0961.wav   7095
test_0961.wav
test_0531.wav   7096
test_0531.wav
test_0763.wav   7097
test_0763.wav
test_0774.wav   7098
test_0774.wav
test_0090.wav   7099
test_0090.wav
test_0893.wav   7100
test_0893.wav
test_0491.wav   7101

test_0786.wav   7308
test_0786.wav
test_0766.wav   7309
test_0766.wav
test_0704.wav   7310
test_0704.wav
test_0722.wav   7311
test_0722.wav
test_0646.wav   7312
test_0646.wav
test_0131.wav   7313
test_0131.wav
test_0376.wav   7314
test_0376.wav
test_0545.wav   7315
test_0545.wav
test_0754.wav   7316
test_0754.wav
test_0897.wav   7317
test_0897.wav
test_0967.wav   7318
test_0967.wav
test_0775.wav   7319
test_0775.wav
test_0651.wav   7320
test_0651.wav
test_0626.wav   7321
test_0626.wav
test_0489.wav   7322
test_0489.wav
test_0613.wav   7323
test_0613.wav
test_0619.wav   7324
test_0619.wav
test_0120.wav   7325
test_0120.wav
test_0364.wav   7326
test_0364.wav
test_0862.wav   7327
test_0862.wav
test_0500.wav   7328
test_0500.wav
test_0527.wav   7329
test_0527.wav
test_0969.wav   7330
test_0969.wav
test_0907.wav   7331
test_0907.wav
test_0125.wav   7332
test_0125.wav
test_0677.wav   7333
test_0677.wav
test_0631.wav   7334
test_0631.wav
test_0995.wav   7335
test_0995.wav
test_0529.wav   7336

In [35]:
from sklearn.svm import SVC
clf = SVC()
clf.fit(train_data,Train_labels)

/home/saieswar/anaconda2/envs/cs633/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [21]:
with open('data.pickle', 'rb') as handle:
    b = pickle.load(handle)

In [23]:
len(b['train_1691.wav'])

590

In [133]:
train_data   = []
train_labels = []
test_data    = []
test_labels  = []
valid_data   = []
valid_labels = []

for key in b.keys():
    if(key.startswith("train")):
        train_data.append(b[key])
        train_labels.append(labels[key])
    elif(key.startswith("test")):
        test_data.append(b[key])
        test_labels.append(labels[key])
    else:
        valid_data.append(b[key])
        valid_labels.append(labels[key])

In [134]:
len(train_data)+len(test_data)+len(valid_data)

7341

In [135]:
from sklearn.decomposition import PCA
pca = PCA(n_components=7)
pca.fit(np.array(train_data))
PCA(copy=True, iterated_power='auto', n_components=200, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)
print(pca.explained_variance_ratio_)
train_pca = pca.transform(np.array(train_data))


[0.89831724 0.02960151 0.01935613 0.00971597 0.00752077 0.00634984
 0.0053992 ]


In [136]:
clf = SVC()
clf.fit(train_pca,train_labels)

/home/saieswar/anaconda2/envs/cs633/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [137]:
test_pca = pca.transform(np.array(test_data))

In [138]:
clf.score(test_pca,test_labels)

0.6916916916916916

In [146]:
clf1 = RandomForestClassifier(n_estimators=1000, max_depth = 10000, n_jobs = -1,\
                             oob_score = True, min_samples_split = 10)

clf1.fit(train_pca,train_labels)

clf1.score(test_pca,test_labels)

0.6446446446446447

In [140]:
test_pca[0]

array([-1195.63067303,   201.2608784 ,   -28.47491958,   -37.28150053,
         -16.57313925,   -11.93252325,    13.25792148])

In [62]:
predict_test = clf.predict(test_pca)

In [65]:
for i,j in zip(predict_test,test_labels):
    if(i!=1):
        print("predicted_value ",i,"actual value",j)

In [47]:
valid_pca = pca.transform(np.array(valid_data))

In [48]:
clf.score(valid_pca,valid_labels)

0.6746666666666666

In [67]:
train_data_s   = []
train_labels_s = []
test_data_s    = []
test_labels_s  = []
valid_data_s   = []
valid_labels_s = []
for i,j in zip(train_data, train_labels):
    if(j!=1):
        train_data_s.append(i)
        train_labels_s.append(j)
for i,j in zip(test_data, test_labels):
    if(j!=1):
        test_data_s.append(i)
        test_labels_s.append(j)
for i,j in zip(valid_data, valid_labels):
    if(j!=1):
        valid_data_s.append(i)
        valid_labels_s.append(j)

In [70]:
from sklearn.decomposition import PCA
pca = PCA(n_components=7)
pca.fit(np.array(train_data_s))
PCA(copy=True, iterated_power='auto', n_components=200, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)
print(pca.explained_variance_ratio_)
train_pca_s = pca.transform(np.array(train_data_s))

[0.90574966 0.02797761 0.0150096  0.00965833 0.00712017 0.00653941
 0.00538284]


In [71]:
clf = SVC()
clf.fit(train_pca_s,train_labels_s)

/home/saieswar/anaconda2/envs/cs633/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [72]:
test_pca_s = pca.transform(np.array(test_data_s))

In [74]:
print(len(test_labels_s))

308


In [73]:
clf.score(test_pca_s,test_labels_s)

0.7045454545454546

Spliting data into classes

In [78]:
train_data_0   = []
train_labels_0 = []
test_data_0    = []
test_labels_0  = []
valid_data_0   = []
valid_labels_0 = []

#############################################
train_data_1   = []
train_labels_1 = []
test_data_1    = []
test_labels_1  = []
valid_data_1   = []
valid_labels_1 = []

#################################

train_data_2   = []
train_labels_2 = []
test_data_2    = []
test_labels_2  = []
valid_data_2   = []
valid_labels_2 = []

###############################
train_data_3   = []
train_labels_3 = []
test_data_3    = []
test_labels_3  = []
valid_data_3   = []
valid_labels_3 = []
############################
for i,j in zip(train_data, train_labels):
    if(j==0):
        train_data_0.append(i)
        train_labels_0.append(j)
    elif (j==1):
        train_data_1.append(i)
        train_labels_1.append(j)
    elif (j==2):
        train_data_2.append(i)
        train_labels_2.append(j)
    else:
        train_data_3.append(i)
        train_labels_3.append(j)

for i,j in zip(test_data, test_labels):
    if(j==0):
        test_data_0.append(i)
        test_labels_0.append(j)
    elif (j==1):
        test_data_1.append(i)
        test_labels_1.append(j)
    elif (j==2):
        test_data_2.append(i)
        test_labels_2.append(j)
    else:
        test_data_3.append(i)
        test_labels_3.append(j)

for i,j in zip(valid_data, valid_labels):
    if(j==0):
        valid_data_0.append(i)
        valid_labels_0.append(j)
    elif (j==1):
        valid_data_1.append(i)
        valid_labels_1.append(j)
    elif (j==2):
        valid_data_2.append(i)
        valid_labels_2.append(j)
    else:
        valid_data_3.append(i)
        valid_labels_3.append(j)


In [ ]:
mapping ={"angry":0,"neutral":1,"happy":2,"sad":3}

In [88]:
train_data_2_class = train_data_0 + train_data_3
test_data_2_class = test_data_0 + test_data_3
train_labels_2_class = train_labels_0 + train_labels_3
test_labels_2_class = test_labels_0 + test_labels_3


In [110]:
valid_data_2_class = valid_data_0 + valid_data_3
valid_labels_2_class = valid_labels_0 + valid_labels_3

In [93]:
clf = SVC()
clf.fit(train_data_2_class,train_labels_2_class)
clf.score(test_data_2_class,test_labels_2_class)

/home/saieswar/anaconda2/envs/cs633/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.8461538461538461

In [100]:
predict_test_labels_class = clf.predict(test_data_2_class)
count = 0
for i,j in zip(predict_test_labels_class,test_labels_2_class):
    if(i==j):
        count+=1;

In [102]:
len(test_data)

0.8461538461538461

In [97]:
len(pre)

91

In [105]:
from sklearn.ensemble import RandomForestClassifier
clf1 = RandomForestClassifier(n_estimators=300, max_depth = 50, n_jobs = -1,\
                             oob_score = True, min_samples_split = 10)

clf1.fit(train_data_2_class, train_labels_2_class)

clf1.score(test_data_2_class,test_labels_2_class)

0.8351648351648352

In [106]:
predict_11 = clf1.predict(test_data_2_class)

In [111]:
valid_data_2_class = valid_data_0 + valid_data_3
valid_labels_2_class = valid_labels_0 + valid_labels_3

In [112]:

clf1.score(valid_data_2_class,valid_labels_2_class)

0.8764044943820225

In [116]:
train_data_21_class = train_data_1 + train_data_2
test_data_21_class = test_data_1 + test_data_2
train_labels_21_class = train_labels_1 + train_labels_2
test_labels_21_class = test_labels_1 + test_labels_2
valid_data_21_class = valid_data_1 + valid_data_2
valid_labels_21_class = valid_labels_1 + valid_labels_2

from sklearn.ensemble import RandomForestClassifier
clf1 = RandomForestClassifier(n_estimators=300, max_depth = 50, n_jobs = -1,\
                             oob_score = True, min_samples_split = 10)

clf1.fit(train_data_21_class, train_labels_21_class)

clf1.score(test_data_21_class,test_labels_21_class)

0.763215859030837

In [132]:
clf2 = SVC()
clf2.fit(train_data_21_class,train_labels_21_class)
clf2.score(test_data_21_class,test_labels_21_class)

/home/saieswar/anaconda2/envs/cs633/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.7610132158590308

In [117]:
predict_21 = clf1.predict(test_data_21_class)